# Computer Vision in JupyterHub
#### *Difficulty: Intermediate*

### 🌎 Overview
In this quickstart, we'll train and run [**YOLOv8**](https://docs.ultralytics.com/), an object detection model, on an open-source dataset from [**Roboflow**](https://roboflow.com/) solely within this JupyterHub notebook!

### 🧠 Prerequisites
- Jupyter Notebooks
- Python
- Basic CLI commands

### ✔ Learning Outcomes
1. Grab an open-source dataset from Roboflow (API)
2. Use Ultralytics' YOLOv8 library to train and test models
3. Create a model fine-tuned on a dataset
4. Test the fine-tuned model on an example video

### 
💾 Resource Requirements
<table align="left">
    <tr>
        <th>Option</th>
        <th>Selection</th>
    </tr>
    <tr>
        <td>GPU type</td>
        <td>L40</td>
    </tr>
    <tr>
        <td>GPUs</td>
        <td>1</td>
    </tr>
    <tr>
        <td>CPU</td>
        <td>2</td>
    </tr>
    <tr>
        <td>RAM</td>
        <td>12</td>
    </tr>
    <tr>
        <td>Image</td>
        <td>Stack PRP</td>
    </tr>
</table>

### 1. 📚 Libraries
In order to make an API call to Roboflow, use YOLOv8, and view results, we'll need to install some external Python libraries

In [1]:
# Install ultralytics, open-cv, and roboflow
! pip install ultralytics==8.0.196 opencv-python==4.8.0.76 roboflow --quiet

# Import external libs to our notebook
from IPython.display import display, Image
import ultralytics
from roboflow import Roboflow
import requests
import json
import pprint
import os

In [2]:
# Check that ultralytics is working and recognizes the notebook's virtual resources
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.11.5 torch-2.0.0+cu117 CUDA:0 (NVIDIA L40, 45373MiB)
Setup complete ✅ (24 CPUs, 503.4 GB RAM, 615.2/799.6 GB disk)


### 2. 📊 Roboflow Dataset
Next, we'll get our Roboflow API key, then download an open-source dataset from Roboflow to be used by our object detection model

#### 2-1. Roboflow API Key
To get our own Roboflow API key, follow the steps below:
1. Log into or create a free Roboflow account [here](https://app.roboflow.com/login)
2. Click on profile > API Keys
3. Copy Private API Key
4. Paste the key into the below variable

In [3]:
# Paste your Roboflow API key here (don't share with others!)
api_key = "0nTlF4AD6oobdevxJNHS"

#### 2-2. Dataset Download
After we've obtained our API key and pasted it into the above Python variable, we'll download an example dataset and show its basic structure

In [4]:
# Set file path shortcuts
HOME = os.getcwd()
DATA_PATH = f"{HOME}/pickleball-frames-1"
YAML_PATH = f"{DATA_PATH}/data.yaml"

# Download the dataset from Roboflow using the API key
rf = Roboflow(api_key=api_key)
project = rf.workspace("pickleballtombradyreachout").project("pickleball-frames")
version = project.version(1)
dataset = version.download("yolov8")

# Print the number of samples
train_dir = f"{DATA_PATH}/train/images"
valid_dir = f"{DATA_PATH}/valid/images"
test_dir = f"{DATA_PATH}/test/images"

# Get the number of images in each subset
train_count = len([file for file in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, file))])
valid_count = len([file for file in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, file))])
test_count = len([file for file in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, file))])

print(f"\nTrain Set: {train_count} Images")
print(f"Valid Set: {valid_count} Images")
print(f"Test Set: {test_count} Images")

loading Roboflow workspace...
loading Roboflow project...

Train Set: 456 Images
Valid Set: 94 Images
Test Set: 93 Images


### 3. 🦾 Training
Once we've downloaded our dataset, we'll fine-tune the YOLOv8 model on this data

In [ ]:
# Flag to protect train/validation/testing from auto-running
commence = True

# Train model (replace with yolov8x.pt for performance/time comparison)
if commence:
    ! yolo task=detect mode=train model=yolov8s.pt data={YAML_PATH} imgsz=800 epochs=25 plots=True workers=0
else:
    print(f"Model has been trained. Results at {HOME}/runs/detect")

New https://pypi.org/project/ultralytics/8.3.90 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.5 torch-2.0.0+cu117 CUDA:0 (NVIDIA L40, 45373MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/jovyan/pickleball-frames-1/data.yaml, epochs=25, patience=50, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, a

### 4. 🎬 Inference
Finally, we'll take our fine-tuned model and test it on a short video

#### 4-1. Test Video
The below command uses the `wget` tool to download a short video of a pro pickleball match which we'll feed to the model we trained above

In [ ]:
# Download an example video from Google Drive
! wget -O test-video.mp4 "https://drive.usercontent.google.com/download?id=1kt3wMNvjxYWqIUeTvk8ZjYbtvLPvvumj&export=download&authuser=0&confirm=yes"

#### 4.2. Object Tracking
The following code uses the fine-tuned model to perform object detection (tracking players) on the test video

In [ ]:
!pip install ultralytics

from PIL import Image
from ultralytics import YOLO
import cv2
import os

# Load the fine-tuned model
# Set file path shortcuts
HOME = os.getcwd()
DATA_PATH = f"{HOME}/pickleball-frames-1"
YAML_PATH = f"{DATA_PATH}/data.yaml"
model_path = f'{HOME}/runs/detect/train/weights/best.pt'
model = YOLO(model_path)

# Open the video file
input_video = f"{HOME}/test-video.mp4"
cap = cv2.VideoCapture(input_video)

# Get the frame rate and size
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a video writer
out = cv2.VideoWriter(f'{HOME}/test-output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Loop over the frames
while cap.isOpened():
    # Read a frame
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame
    results = model(frame)

    # Plot the results on the frame
    im_array = results[0].plot()

    # Write the frame to the output file
    out.write(im_array)
    # Wait for a key press or end of video
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture, writer and destroy the window
cap.release()
out.release()

print(f"Model has produced output video. Results at {HOME}/test-output.mp4")